In [2]:
########################################################################
# import default libraries
########################################################################
import os
import sys
import gc
########################################################################


########################################################################
# import additional libraries
########################################################################
import numpy as np
import scipy.stats
# from import

from tqdm import tqdm
try:
    from sklearn.externals import joblib
except:
    import joblib
# original lib
import keras_model_facenet
import pandas as pd
import keras
import tensorflow as tf
import tensorflow_io as tfio
import keras.backend as K
import common as com
from __future__ import division 
import numpy as np
import scipy.signal as sps
import scipy.io.wavfile as wf
############

/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIFN3tsl8StatusOrISt10unique_

In [4]:
########################################################################
# load parameter.yaml
########################################################################
param = com.yaml_load()
########################################################################
saved_weight = os.path.join(param["P_MODELSAVE"], 'dataweights.{epoch:02d}-{val_dense_1_binary_accuracy:.2f}.hdf5')
early_stopping = keras.callbacks.EarlyStopping(monitor = 'loss', patience = 1000)
# model_unet = keras_model.get_model(input_shape=(1,param["feature"]["n_mels"],param["feature"]["n_frames"]), lr = param["fit"]["lr"])

########################################################################
# get data from the list for file paths
########################################################################
def get_label(file_path):
  # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    parts = tf.strings.split(parts[-1], sep='_')
    num = tf.strings.to_number(parts[1], out_type=tf.int64)
    return num

def get_name(file_path):
  # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    parts = tf.strings.split(parts[-1])
    return parts

def file_high(file_name,
                n_mels=128,
                n_fft=1024,
                hop_length=431,
                power=2.0):
    """
    convert file_name to a vector array.

    file_name : str
        target .wav file

    return : numpy.array( numpy.array( float ) )
        vector array
        * dataset.shape = (dataset_size, feature_vector_length)
    """
    # Dictionary
    table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
    keys=tf.constant([2, 3, 5, 7, 1, 4, 6, 8]),
    values=tf.constant([0, 1, 2, 3, 0, 1, 2, 3]),
    ),
    default_value=tf.constant(-1),
    name="class_weight"
    )
    label1 = get_label(file_name)
    # generate melspectrogram using tf.io
    file_contents = tf.io.read_file(file_name)
    data, sr = tf.audio.decode_wav(file_contents, desired_channels=1)
    data = tf.squeeze(data, axis=[-1])
    n = tf.split(data, 5)
    # label1 = tf.one_hot(int(label1), 8)
    label1 = tf.one_hot(table.lookup(int(label1)), 4)
    label2 = tf.one_hot(0, 1)
    return n[0],n[1],n[2],n[3],n[4], label2, tf.concat([label1, label2], 0)


def file_low(file_name):
    """
    convert file_name to a vector array.

    file_name : str
        target .wav file

    return : numpy.array( numpy.array( float ) )
        vector array
        * dataset.shape = (dataset_size, feature_vector_length)
    """
    table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
    keys=tf.constant([2, 3, 5, 7, 1, 4, 6, 8]),
    values=tf.constant([0, 1, 2, 3, 0, 1, 2, 3]),
    ),
    default_value=tf.constant(-1),
    name="class_weight"
    )
    label1 = get_label(file_name)
    # generate melspectrogram using tf.io
    file_contents = tf.io.read_file(file_name)
    data, sr = tf.audio.decode_wav(file_contents, desired_channels=1)
    data = tf.squeeze(data, axis=[-1])
    n = tf.split(data, 5)
    # label1 = tf.one_hot(int(label1), 8)
    label1 = tf.one_hot(table.lookup(int(label1)), 4)
    label2 = tf.one_hot(1, 1)
    return n[0],n[1],n[2],n[3],n[4], label2, tf.concat([label1, label2], 0)


def file_ano(file_name):
    """
    convert file_name to a vector array.

    file_name : str
        target .wav file

    return : numpy.array( numpy.array( float ) )
        vector array
        * dataset.shape = (dataset_size, feature_vector_length)
    """
    table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
    keys=tf.constant([2, 3, 5, 7, 1, 4, 6, 8]),
    values=tf.constant([0, 1, 2, 3, 0, 1, 2, 3]),
    ),
    default_value=tf.constant(-1),
    name="class_weight"
    )
    # generate melspectrogram using tf.io
    file_contents = tf.io.read_file(file_name)
    # label1 = tf.one_hot(int(label1), 8)
    label7 = tf.one_hot(3, 4)
    label5 = tf.one_hot(2, 4)
    label3 = tf.one_hot(1, 4)
    label2 = tf.one_hot(0, 4)
    
    
    label = tf.one_hot(1, 1)
    data, sr = tf.audio.decode_wav(file_contents)
    data = tf.transpose(data)
    return data[0], data[1], data[2], data[3], label7, label5, label2, label3, label 
    # return data, label2, tf.concat([label1, label2], 0)


def to_mel(y, machine, section,
        anomaly = False,
        n_mels=128,
        n_fft=2823,
        hop_length=347,
        power=2.0,
        sr=22050):
    
    spectrogram = tfio.audio.spectrogram(y, nfft=n_fft, window=n_fft, stride=hop_length)
    mel_spectrogram = tfio.audio.melscale(spectrogram, sr, mels=n_mels, fmin=0, fmax=11025, name=None)
    dbscale_mel_spectrogram = tfio.audio.dbscale(mel_spectrogram, top_db=120)
    if anomaly == True:
        dbscale_mel_spectrogram = tf.transpose(dbscale_mel_spectrogram)
    dbscale_mel_spectrogram = tf.expand_dims(dbscale_mel_spectrogram, axis=2)
    return dbscale_mel_spectrogram, {'affine': machine, 'linear': section}

In [5]:
target_dir1 = "dev_data/high_noise"
target_dir2 = "dev_data/low_noise"
target_dir3 = "dev_data/mid_noise"
target_dir4 = "dev_data/anomalous"
target_dir5 = "dev_data_doa/combined_ground_truth/train"
# loop of the base directory
print("\n===========================")

# generate dataset
print("============== DATASET_GENERATOR ==============")

# get file list for all sections
# all values of y_true are zero in training
# Normal
pos1 = tf.data.Dataset.list_files(f"{target_dir1}/train/section_02*.wav")
pos2 = tf.data.Dataset.list_files(f"{target_dir1}/train/section_03*.wav")
pos3 = tf.data.Dataset.list_files(f"{target_dir3}/train/section_05*.wav")
pos4 = tf.data.Dataset.list_files(f"{target_dir3}/train/section_07*.wav")

# Other Noise
pos5 = tf.data.Dataset.list_files(f"{target_dir2}/train/*.wav")
pos6 = tf.data.Dataset.list_files(f"{target_dir1}/train/section_04*.wav")

# Anomaly
real_anomalous = tf.data.Dataset.list_files(f"{target_dir5}/*.wav")

# pos1 = pos1.concatenate(pos_l).concatenate(pos_ll).concatenate(pos_lll).shuffle(3000)
# pos2 = pos2.concatenate(pos_l2).shuffle(5000)
normal = pos1.concatenate(pos2).concatenate(pos3).concatenate(pos4).shuffle(5000)
anomaly = pos5.concatenate(pos6).shuffle(5000)
anomaly2 = real_anomalous.shuffle(5000)
print(normal.cardinality().numpy() ,anomaly.cardinality().numpy())
train_size_anomaly = int(0.8 * anomaly.cardinality().numpy())
train_size_normal = int(0.8 * normal.cardinality().numpy())
train_size_real_anomaly = int(0.8 * anomaly2.cardinality().numpy())

print(train_size_normal, train_size_anomaly, train_size_real_anomaly) 

train_dataset_normal = normal.take(train_size_normal)
train_dataset_anomaly = anomaly.take(train_size_anomaly)
train_dataset_real_anomaly = anomaly2.take(train_size_real_anomaly)

val_dataset_normal = normal.skip(train_size_normal)
val_dataset_anomaly = anomaly.skip(train_size_anomaly)
val_dataset_real_anomaly = anomaly2.skip(train_size_real_anomaly)



============== DATASET_GENERATOR ==============
4251 4252
3400 3401 3261


In [6]:

############## TRAIN

# High Noise
dataset_high = train_dataset_normal.map(file_high).map(lambda a, b, c, d, e, x, y: (a, x, y, False))
dataset_high2 = train_dataset_normal.map(file_high).map(lambda a, b, c, d, e, x, y: (b, x, y, False))
dataset_high3 = train_dataset_normal.map(file_high).map(lambda a, b, c, d, e, x, y: (c, x, y, False))
dataset_high4 = train_dataset_normal.map(file_high).map(lambda a, b, c, d, e, x, y: (d, x, y, False))
dataset_high5 = train_dataset_normal.map(file_high).map(lambda a, b, c, d, e, x, y: (e, x, y, False))

dataset_train_high = dataset_high.concatenate(dataset_high2).concatenate(dataset_high3).concatenate(dataset_high4).concatenate(dataset_high5)

# Low Noise
dataset_mid = train_dataset_anomaly.map(file_low).map(lambda a, b, c, d, e, x, y: (a, x, y, False))
dataset_mid2 = train_dataset_anomaly.map(file_low).map(lambda a, b, c, d, e, x, y: (b, x, y, False))
dataset_mid3 = train_dataset_anomaly.map(file_low).map(lambda a, b, c, d, e, x, y: (c, x, y, False))
dataset_mid4 = train_dataset_anomaly.map(file_low).map(lambda a, b, c, d, e, x, y: (d, x, y, False))
dataset_mid5 = train_dataset_anomaly.map(file_low).map(lambda a, b, c, d, e, x, y: (e, x, y, False))

dataset_train_mid = dataset_mid.concatenate(dataset_mid2).concatenate(dataset_mid3).concatenate(dataset_mid4).concatenate(dataset_mid5)

# Anomaly
dataset_ano1 = train_dataset_real_anomaly.map(file_ano).map(lambda a, b, c, d, x, y, z, i, j: (a, j, tf.concat([x, j], 0), True))
dataset_ano2 = train_dataset_real_anomaly.map(file_ano).map(lambda a, b, c, d, x, y, z, i, j: (b, j, tf.concat([y, j], 0), True))
dataset_ano3 = train_dataset_real_anomaly.map(file_ano).map(lambda a, b, c, d, x, y, z, i, j: (c, j, tf.concat([z, j], 0), True))
dataset_ano4 = train_dataset_real_anomaly.map(file_ano).map(lambda a, b, c, d, x, y, z, i, j: (d, j, tf.concat([i, j], 0), True))

dataset_ano = dataset_ano1.concatenate(dataset_ano2).concatenate(dataset_ano3).concatenate(dataset_ano4)

# All
dataset_train_high = dataset_train_high.concatenate(dataset_ano).concatenate(dataset_train_mid)


############# VAL
# High Noise
dataset_high = val_dataset_normal.map(file_high).map(lambda a, b, c, d, e, x, y: (a, x, y, False))
dataset_high2 = val_dataset_normal.map(file_high).map(lambda a, b, c, d, e, x, y: (b, x, y, False))
dataset_high3 = val_dataset_normal.map(file_high).map(lambda a, b, c, d, e, x, y: (c, x, y, False))
dataset_high4 = val_dataset_normal.map(file_high).map(lambda a, b, c, d, e, x, y: (d, x, y, False))
dataset_high5 = val_dataset_normal.map(file_high).map(lambda a, b, c, d, e, x, y: (e, x, y, False))

dataset_val_high = dataset_high.concatenate(dataset_high2).concatenate(dataset_high3).concatenate(dataset_high4).concatenate(dataset_high5)

# Low Noise
dataset_low = val_dataset_anomaly.map(file_low).map(lambda a, b, c, d, e, x, y: (a, x, y, False))
dataset_low2 = val_dataset_anomaly.map(file_low).map(lambda a, b, c, d, e, x, y: (b, x, y, False))
dataset_low3 = val_dataset_anomaly.map(file_low).map(lambda a, b, c, d, e, x, y: (c, x, y, False))
dataset_low4 = val_dataset_anomaly.map(file_low).map(lambda a, b, c, d, e, x, y: (d, x, y, False))
dataset_low5 = val_dataset_anomaly.map(file_low).map(lambda a, b, c, d, e, x, y: (e, x, y, False))

dataset_val_low = dataset_low.concatenate(dataset_low2).concatenate(dataset_low3).concatenate(dataset_low4).concatenate(dataset_low5)

# Anomaly
dataset_ano1 = val_dataset_real_anomaly.map(file_ano).map(lambda a, b, c, d, x, y, z, i, j: (a, j, tf.concat([x, j], 0), True))
dataset_ano2 = val_dataset_real_anomaly.map(file_ano).map(lambda a, b, c, d, x, y, z, i, j: (b, j, tf.concat([y, j], 0), True))
dataset_ano3 = val_dataset_real_anomaly.map(file_ano).map(lambda a, b, c, d, x, y, z, i, j: (c, j, tf.concat([z, j], 0), True))
dataset_ano4 = val_dataset_real_anomaly.map(file_ano).map(lambda a, b, c, d, x, y, z, i, j: (d, j, tf.concat([i, j], 0), True))
dataset_ano = dataset_ano1.concatenate(dataset_ano2).concatenate(dataset_ano3).concatenate(dataset_ano4)
# All
dataset_val_high = dataset_val_high.concatenate(dataset_val_low).concatenate(dataset_ano)


# FINAL
BATCH_SIZE=64
train_dataset = dataset_train_high.map(to_mel).cache("tmp/cache").shuffle(50000).batch(BATCH_SIZE).prefetch(1)
val_dataset = dataset_val_high.map(to_mel).cache("tmp2/cache").shuffle(12000).batch(BATCH_SIZE).prefetch(1)

In [11]:
def BinaryCrossEntropy_machine(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    term_0 = (1 - y_true) * K.log(1 - y_pred + K.epsilon())  
    term_1 = y_true * K.log(y_pred + K.epsilon())
    result = -K.mean(term_0 + term_1, axis=0)
    return result

def binary_crossentropy_test(y_true, y_pred):
    batch_size = K.shape(y_true)[0]
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    term_0 = (1 - y_true[:,:-1]) * K.log(1 - y_pred + K.epsilon())  
    term_1 = y_true[:,:-1] * K.log(y_pred + K.epsilon())
    size = tf.cast(tf.size(term_0 + term_1), dtype=tf.float32) * tf.cast(tf.size(y_true[:,-1]), dtype=tf.float32)
    machine_true = tf.reshape(y_true[:,-1], [batch_size, 1])
    result = -K.mean(K.mean((term_0 + term_1)*machine_true, axis=0))
    return result

def BinaryCrossEntropy_section(y_true, y_pred): 
    batch_size = K.shape(y_true)[0]
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    term_0 = (1 - y_true[:,:-1]) * K.log(1 - y_pred + K.epsilon())  
    term_1 = y_true[:,:-1] * K.log(y_pred + K.epsilon())
    size = tf.cast(tf.size(term_0 + term_1), dtype=tf.float32) * tf.cast(tf.size(y_true[:,-1]), dtype=tf.float32)
    machine_true = tf.reshape(y_true[:,-1], [batch_size, 1])
    result = -tf.math.divide_no_nan(K.sum(K.sum((term_0 + term_1)*machine_true, axis=0)), (size + K.epsilon()))
    return result

def binary_accuracy_section(y_true, y_pred):
    batch_size = K.shape(y_true)[0]
    machine_true = tf.reshape(y_true[:,-1], [batch_size, 1])
    return K.mean(K.equal(y_true[:,:-1]*machine_true, K.round(y_pred)), axis=-1)

class DecayHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.lr = []
        self.wd = []
    def on_batch_end(self, batch, logs={}):
        self.lr.append(self.model.optimizer.lr(self.model.optimizer.iterations))
        self.wd.append(self.model.optimizer.weight_decay)
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 20, 30 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-4

    if epoch >= 10:
        lr *= 1e-1
    elif epoch >= 20:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def wd_schedule(epoch):
    """Weight Decay Schedule
    Weight decay is scheduled to be reduced after 20, 30 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        wd (float32): weight decay
    """
    wd = 1e-5

    if epoch >= 20:
        wd *= 1e-1
    elif epoch >= 10:
        wd *= 1e-1
    print('Weight decay: ', wd)
    return wd

tb_callback = tf.keras.callbacks.TensorBoard(os.path.join('logs', 'adamw'),
                                             profile_batch=0)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

def BinaryCrossEntropy_machine(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    term_0 = (1 - y_true) * K.log(1 - y_pred + K.epsilon())  
    term_1 = y_true * K.log(y_pred + K.epsilon())
    result = -K.mean(term_0 + term_1, axis=0)
    return result

def binary_crossentropy_test(y_true, y_pred):
    batch_size = K.shape(y_true)[0]
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    term_0 = (1 - y_true[:,:-1]) * K.log(1 - y_pred + K.epsilon())  
    term_1 = y_true[:,:-1] * K.log(y_pred + K.epsilon())
    size = tf.cast(tf.size(term_0 + term_1), dtype=tf.float32) * tf.cast(tf.size(y_true[:,-1]), dtype=tf.float32)
    machine_true = tf.reshape(y_true[:,-1], [batch_size, 1])
    result = -K.mean(K.mean((term_0 + term_1)*machine_true, axis=0))
    return result

def categorical_cross_entropy(y_true, y_pred):
    batch_size = K.shape(y_true)[0]
    # y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    term_1 = y_true[:,:-1] * K.log(y_pred + K.epsilon())
    machine_true = tf.reshape(y_true[:,-1], [batch_size, 1])
    result = -K.mean(K.sum(term_1*machine_true, axis=0))
    return result


def BinaryCrossEntropy_section(y_true, y_pred): 
    batch_size = K.shape(y_true)[0]
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    term_0 = (1 - y_true[:,:-1]) * K.log(1 - y_pred + K.epsilon())  
    term_1 = y_true[:,:-1] * K.log(y_pred + K.epsilon())
    size = tf.cast(tf.size(term_0 + term_1), dtype=tf.float32) * tf.cast(tf.size(y_true[:,-1]), dtype=tf.float32)
    machine_true = tf.reshape(y_true[:,-1], [batch_size, 1])
    result = -tf.math.divide_no_nan(K.sum(K.sum((term_0 + term_1)*machine_true, axis=0)), (size + K.epsilon()))
    return result

def binary_accuracy_section(y_true, y_pred):
    batch_size = K.shape(y_true)[0]
    machine_true = tf.reshape(y_true[:,-1], [batch_size, 1])
    return K.mean(K.equal(y_true[:,:-1]*machine_true, K.round(y_pred)), axis=-1)

def categorical_accuracy_section(y_true, y_pred, class_to_ignore=0):
    batch_size = K.shape(y_true)[0]
    machine_true = tf.reshape(y_true[:,-1], [batch_size, 1])
    return metrics_utils.sparse_categorical_matches(
        tf.math.argmax(y_true*machine_true, axis=-1), y_pred
    )


In [ ]:
# number of vectors for each wave file
# train model


initial_learning_rate = 0.0005
final_learning_rate = 0.00001
learning_rate_decay_factor = (final_learning_rate / initial_learning_rate)**(1/200)
steps_per_epoch = int(736)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=initial_learning_rate,
                decay_steps=steps_per_epoch,
                decay_rate=learning_rate_decay_factor,
                staircase=True)


print("============== MODEL TRAINING ==============")
model = keras_model_facenet.get_model((128,128,1), num_class=4)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005), loss = 
              {"affine":"binary_crossentropy", "linear":binary_crossentropy_test},
              metrics= {"affine"  :'binary_accuracy', "linear": binary_accuracy_section},
              loss_weights=[1, 1])

model.summary()

# Load Checkpoint

history = model.fit(train_dataset,
                    epochs=300,
                    verbose=1,
                    validation_data = val_dataset,
                    callbacks = [checkpoint])


pd.DataFrame.from_dict(model.history.history).to_csv(f'history_{machine_type}_2.csv',index=False)


model.save(model_file_path)
com.logger.info("save_model -> {}".format(model_file_path))

print("============== END TRAINING ==============")


gc.collect()


============== MODEL TRAINING ==============
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 64)   640         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                 

KeyboardInterrupt: 

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
gmm = GaussianMixture(n_components=1)

In [ ]:
model = keras.models.load_model("saved_models_facenet/model_35+18.h5", custom_objects={'BinaryCrossEntropy_machine': BinaryCrossEntropy_machine, 'BinaryCrossEntropy_section': BinaryCrossEntropy_section })
model_output = model.get_layer('re_lu').output
m = keras.Model(inputs=model.input, outputs=model_output)

In [ ]:
def get_ano(file_path):
  # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    parts = tf.strings.split(parts[-1], sep='_')
    return parts[4]

def file_to_vectors(file_name):
    """
    convert file_name to a vector array.

    file_name : str
        target .wav file

    return : numpy.array( numpy.array( float ) )
        vector array
        * dataset.shape = (dataset_size, feature_vector_length)
    """
    table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
    keys=tf.constant([2, 3, 4, 5, 1, 6, 7, 8]),
    values=tf.constant([0, 1, 2, 3, 0 , 1, 2, 3]),
    ),
    default_value=tf.constant(-1),
    name="class_weight"
    )
    name = get_name(file_name)
    # generate melspectrogram using tf.io
    file_contents = tf.io.read_file(file_name)
    data, sr = tf.audio.decode_wav(file_contents, desired_channels=1)
    data = tf.squeeze(data, axis=[-1])
    n = tf.split(data, 5)
    return n[0],n[1],n[2],n[3],n[4], name

def file_to_vectors2(file_name):
    """
    convert file_name to a vector array.

    file_name : str
        target .wav file

    return : numpy.array( numpy.array( float ) )
        vector array
        * dataset.shape = (dataset_size, feature_vector_length)
    """
    table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
    keys=tf.constant([2, 3, 4, 5, 1, 6, 7, 8]),
    values=tf.constant([0, 1, 2, 3, 0 , 1, 2, 3]),
    ),
    default_value=tf.constant(-1),
    name="class_weight"
    )
    name = get_name(file_name)
    # generate melspectrogram using tf.io
    file_contents = tf.io.read_file(file_name)
    data, sr = tf.audio.decode_wav(file_contents, desired_channels=1)
    data = tf.squeeze(data, axis=[-1])
    n = tf.split(data, 5)
    return n[0],n[1],n[2],n[3],n[4], name




def to_mel2(y, name,
        n_mels=128,
        n_fft=2823,
        hop_length=347,
        power=2.0,
        sr=22050):
    spectrogram = tfio.audio.spectrogram(y, nfft=n_fft, window=n_fft, stride=hop_length)
    mel_spectrogram = tfio.audio.melscale(spectrogram, sr, mels=n_mels, fmin=0, fmax=11025, name=None)
    dbscale_mel_spectrogram = tfio.audio.dbscale(mel_spectrogram, top_db=120)
    dbscale_mel_spectrogram = tf.expand_dims(dbscale_mel_spectrogram, axis=2)
    return dbscale_mel_spectrogram, name


pos5 = tf.data.Dataset.list_files(f"dev_data/low_noise/test/*.wav")
dataset_target = pos5.map(file_to_vectors2).map(lambda a, b, c, d, e, x: (a, x))
dataset_target2 = pos5.map(file_to_vectors2).map(lambda a, b, c, d, e, x: (b, x))
dataset_target3 = pos5.map(file_to_vectors2).map(lambda a, b, c, d, e, x: (c, x))
dataset_target4 = pos5.map(file_to_vectors2).map(lambda a, b, c, d, e, x: (d, x))
dataset_target5 = pos5.map(file_to_vectors2).map(lambda a, b, c, d, e, x: (e, x))

dataset_train_low_gmm = dataset_target.concatenate(dataset_target2).concatenate(dataset_target3).concatenate(dataset_target4).concatenate(dataset_target5).map(to_mel2)
# train_gmm_label = dataset_train_low_gmm.map(lambda a, x: x)
# dataset_train_low_gmm = dataset_train_low_gmm.map(lambda a, x: a)

# HIGH
pos6 = tf.data.Dataset.list_files(f"dev_data/high_noise/test/*.wav")
dataset_target = pos6.map(file_to_vectors).map(lambda a, b, c, d, e, x: (a, x))
dataset_target2 = pos6.map(file_to_vectors).map(lambda a, b, c, d, e, x: (b, x))
dataset_target3 = pos6.map(file_to_vectors).map(lambda a, b, c, d, e, x: (c, x))
dataset_target4 = pos6.map(file_to_vectors).map(lambda a, b, c, d, e, x: (d, x))
dataset_target5 = pos6.map(file_to_vectors).map(lambda a, b, c, d, e, x: (e, x))

dataset_train_high_gmm = dataset_target.concatenate(dataset_target2).concatenate(dataset_target3).concatenate(dataset_target4).concatenate(dataset_target5).map(to_mel2)
# train_gmm_label2 = dataset_train_high_gmm.map(lambda a, x: x)
# train_gmm_label = train_gmm_label.concatenate(train_gmm_label2)
# dataset_train_high_gmm = dataset_train_high_gmm.map(lambda a,x: a)


train_gmm = dataset_train_high_gmm.concatenate(dataset_train_low_gmm).batch(1).prefetch(1)
train_label = train_gmm.map(lambda a, x: x)


In [ ]:
import seaborn as sns
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, verbose=1, random_state=123,learning_rate='auto')
z = tsne.fit_transform(features)
df = pd.DataFrame()

# train2 = tfds.as_dataframe(train.take(10))


In [ ]:
train_gmm_label = train_gmm_label.batch(1).prefetch(1)


In [ ]:
print(train_gmm)
for x in train_gmm.take(1):
    print(x)

In [ ]:
df = pd.DataFrame()
df["y"] = train_gmm_label
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]
df.to_csv('ibuki_new_correct.csv')  

In [ ]:
# print(features.shape)
df2 = pd.read_csv('ibuki_new_correct.csv')

In [ ]:
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(12,12)})
sns.scatterplot(x="comp-1", y="comp-2",
                data=df).set(title="MNIST data T-SNE projection")
                # palette=sns.color_palette("hls", 4), hue=df2.y.tolist(),

# plt.ylim(-2, 2)
# plt.xlim(-2, 2)
# plt.show()

In [ ]:
import seaborn as sns
from sklearn.manifold import TSNE
model = keras.models.load_model("saved_models_facenet/model_53+008.h5", custom_objects={'BinaryCrossEntropy_machine': BinaryCrossEntropy_machine, 'binary_accuracy_section':binary_accuracy_section, 'BinaryCrossEntropy_section': BinaryCrossEntropy_section })
model_output = model.get_layer('re_lu_15').output

m = keras.Model(inputs=model.input, outputs=model_output)
m.summary()


In [ ]:
feature = m.predict(train_gmm)

In [ ]:
########################################## FOR AUC ROC BASED ON ACCURACY #####################################################
def file_ano2(file_name):
    """
    convert file_name to a vector array.

    file_name : str
        target .wav file

    return : numpy.array( numpy.array( float ) )
        vector array
        * dataset.shape = (dataset_size, feature_vector_length)
    """
    table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
    keys=tf.constant([2, 3, 4, 5, 1, 6, 7, 8]),
    values=tf.constant([0, 1, 2, 3, 0 , 1, 2, 3]),
    ),
    default_value=tf.constant(-1),
    name="class_weight"
    )
    name = get_name(file_name)
    # generate melspectrogram using tf.io
    file_contents = tf.io.read_file(file_name)
    data, sr = tf.audio.decode_wav(file_contents, desired_channels=1)
    data = tf.squeeze(data, axis=[-1])
    # label1 = tf.one_hot(int(label1), 8)
    return data, name

dataset_high = val_dataset1.map(file_to_vectors).map(lambda a, b, c, d, e, x: (a, x))
dataset_high2 = val_dataset1.map(file_to_vectors).map(lambda a, b, c, d, e, x: (b, x))
dataset_high3 = val_dataset1.map(file_to_vectors).map(lambda a, b, c, d, e, x: (c, x))
dataset_high4 = val_dataset1.map(file_to_vectors).map(lambda a, b, c, d, e, x: (d, x))
dataset_high5 = val_dataset1.map(file_to_vectors).map(lambda a, b, c, d, e, x: (e, x))

dataset_val_high = dataset_high.concatenate(dataset_high2).concatenate(dataset_high3).concatenate(dataset_high4).concatenate(dataset_high5).map(to_mel2)

# Anomaly
dataset_ano = val_dataset3.map(file_ano2).map(to_mel2)
dataset_val_high = dataset_val_high.concatenate(dataset_ano)

# Low Noise
dataset_low = val_dataset2.map(file_to_vectors2).map(lambda a, b, c, d, e, x: (a, x))
dataset_low2 = val_dataset2.map(file_to_vectors2).map(lambda a, b, c, d, e, x: (b, x))
dataset_low3 = val_dataset2.map(file_to_vectors2).map(lambda a, b, c, d, e, x: (c, x))
dataset_low4 = val_dataset2.map(file_to_vectors2).map(lambda a, b, c, d, e, x: (d, x))
dataset_low5 = val_dataset2.map(file_to_vectors2).map(lambda a, b, c, d, e, x: (e, x))

dataset_val_low = dataset_low.concatenate(dataset_low2).concatenate(dataset_low3).concatenate(dataset_low4).concatenate(dataset_low5).map(to_mel2)
val_dataset_test = dataset_val_low.concatenate(dataset_val_high).batch(1).prefetch(1)

machine, section = model.predict(val_dataset_test)
df = pd.DataFrame()
df["name"] = val_dataset_test.map(lambda x, y: y)


In [ ]:
df["Machine_pred"] = [x for x in machine]
df["Section_pred"] = [x for x in section]
df.to_csv('AUC_ROC.csv')  

In [ ]:
########################################## TSNE #####################################################
tsne = TSNE(n_components=2, verbose=1, random_state=123, init='pca', learning_rate='auto')
z = tsne.fit_transform(feature)
df = pd.DataFrame()
df["y"] = train_label
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]
df.to_csv('test_only_random_10.csv')  

In [ ]:
import matplotlib.pyplot as plt
df2 = pd.read_csv('test_only_random_10.csv')
sns.set(rc={'figure.figsize':(15,15)})
sns.scatterplot(x="comp-1", y="comp-2",
                palette=sns.color_palette("hls", 4), hue=df2.y.tolist(),
                data=df).set(title="MNIST data T-SNE projection")

In [ ]:
!pip install visualkeras


In [ ]:
import visualkeras
from PIL import ImageFont
import keras
model2 = keras.models.load_model("model_car1.hdf5")
visualkeras.layered_view(model2, legend=True)

In [ ]:
############# VAL
dataset_high = val_dataset1.map(file_high).map(lambda a, b, c, d, e, x, y: (a, 1.0))
dataset_high2 = val_dataset1.map(file_high).map(lambda a, b, c, d, e, x, y: (b, x, y))
dataset_high3 = val_dataset1.map(file_high).map(lambda a, b, c, d, e, x, y: (c, x, y))
dataset_high4 = val_dataset1.map(file_high).map(lambda a, b, c, d, e, x, y: (d, x, y))
dataset_high5 = val_dataset1.map(file_high).map(lambda a, b, c, d, e, x, y: (e, x, y))

dataset_val_high = dataset_high.concatenate(dataset_high2).concatenate(dataset_high3).concatenate(dataset_high4).concatenate(dataset_high5).map(to_mel)

# Anomaly
dataset_ano = val_dataset3.map(file_ano).map(to_mel)
dataset_val_high = dataset_val_high.concatenate(dataset_ano)

# Low Noise
dataset_low = val_dataset2.map(file_low).map(lambda a, b, c, d, e, x, y: (a, x, y))
dataset_low2 = val_dataset2.map(file_low).map(lambda a, b, c, d, e, x, y: (b, x, y))
dataset_low3 = val_dataset2.map(file_low).map(lambda a, b, c, d, e, x, y: (c, x, y))
dataset_low4 = val_dataset2.map(file_low).map(lambda a, b, c, d, e, x, y: (d, x, y))
dataset_low5 = val_dataset2.map(file_low).map(lambda a, b, c, d, e, x, y: (e, x, y))

dataset_val_low = dataset_low.concatenate(dataset_low2).concatenate(dataset_low3).concatenate(dataset_low4).concatenate(dataset_low5).map(to_mel)

In [ ]:
val_dataset